# Gear Inspection System (OpenCV)

This notebook compares an **ideal gear** image (`ideal.png`) with multiple samples (`sample2.png`, `sample3.png`, …)
to automatically detect **broken** or **worn teeth**.

- Red = missing gear parts (broken/worn)
- Blue = extra material (rare)


In [ ]:
import cv2 as cv
import numpy as np
import glob, os
import matplotlib.pyplot as plt

def load_binary(path):
    img = cv.imread(path, cv.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(path)
    img = cv.GaussianBlur(img, (5,5), 0)
    _, th = cv.threshold(img, 0, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)
    if th.mean() > 128:
        th = cv.bitwise_not(th)
    cnts, _ = cv.findContours(th, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    if not cnts:
        return th, (0,0), 0
    c = max(cnts, key=cv.contourArea)
    (x,y), r = cv.minEnclosingCircle(c)
    return th, (int(x), int(y)), int(r)

def translate_to_center(img, center, target_center, size):
    h, w = img.shape
    canvas = np.zeros(size, dtype=img.dtype)
    dx = target_center[0] - center[0]
    dy = target_center[1] - center[1]
    M = np.float32([[1,0,dx],[0,1,dy]])
    shifted = cv.warpAffine(img, M, (w, h), flags=cv.INTER_NEAREST, borderValue=0)
    ch, cw = canvas.shape
    canvas[:h, :w] = shifted[:ch, :cw]
    return canvas

def scale_to_radius(img, center, current_r, target_r):
    if current_r == 0:
        return img
    scale = float(target_r)/float(current_r)
    h, w = img.shape
    M = cv.getRotationMatrix2D(center, 0, scale)
    return cv.warpAffine(img, M, (w, h), flags=cv.INTER_NEAREST, borderValue=0)

def annulus_mask(shape, center, r_min, r_max):
    Y, X = np.ogrid[:shape[0], :shape[1]]
    dist = np.sqrt((X-center[0])**2 + (Y-center[1])**2)
    return ((dist >= r_min) & (dist <= r_max)).astype(np.uint8)*255

def angular_hist(binary, center, r_min, r_max, num_bins=360):
    mask = annulus_mask(binary.shape, center, r_min, r_max)
    ys, xs = np.where(mask==255)
    angles = (np.arctan2(ys-center[1], xs-center[0]) + np.pi)
    vals = binary[ys, xs]//255
    hist = np.zeros(num_bins, dtype=np.float32)
    idx = (angles * (num_bins/(2*np.pi))).astype(int) % num_bins
    np.add.at(hist, idx, vals)
    return hist

def count_regions_from_hist(missing_hist, min_ang_width_deg=6):
    if missing_hist.max() == 0:
        return 0, []
    thr = 0.25 * missing_hist.max()
    mask = (missing_hist >= thr).astype(np.uint8)
    runs = []
    in_run = False
    start = 0
    for i, v in enumerate(mask):
        if v and not in_run:
            in_run = True
            start = i
        elif not v and in_run:
            in_run = False
            runs.append((start, i-1))
    if in_run:
        runs.append((start, len(mask)-1))
    return len(runs), runs

def classify_sample(ideal_bin, sample_bin, ideal_center, ideal_r):
    missing = cv.bitwise_and(ideal_bin, cv.bitwise_not(sample_bin))
    extra = cv.bitwise_and(sample_bin, cv.bitwise_not(ideal_bin))
    r_min = int(ideal_r*0.80)
    r_max = int(ideal_r*0.98)
    miss_hist = angular_hist(missing, ideal_center, r_min, r_max, num_bins=360)
    num_missing, _ = count_regions_from_hist(miss_hist)
    coverage = (miss_hist > 0).sum()/len(miss_hist)
    if coverage > 0.55:
        return "worn", num_missing, missing, extra
    return "broken", num_missing, missing, extra


In [ ]:
ideal_path = "ideal.png"
ideal_bin, ideal_center, ideal_r = load_binary(ideal_path)
h, w = ideal_bin.shape
target_center = (w//2, h//2)
ideal_aligned = translate_to_center(ideal_bin, ideal_center, target_center, (h,w))
ideal_center = target_center

samples = sorted([p for p in glob.glob("sample*.png")])
results = []

for spath in samples:
    samp_bin, samp_center, samp_r = load_binary(spath)
    samp_scaled = scale_to_radius(samp_bin, samp_center, samp_r, ideal_r)
    samp_aligned = translate_to_center(samp_scaled, samp_center, ideal_center, (h,w))
    label, count, missing, extra = classify_sample(ideal_aligned, samp_aligned, ideal_center, ideal_r)
    results.append((os.path.basename(spath), label, count, missing, extra, samp_aligned))
    print(f"{os.path.basename(spath)} -> {label.upper()} teeth: {count}")

In [ ]:
# Show diagnostic overlays inline
for fname, label, count, missing, extra, samp in results:
    overlay = cv.cvtColor(samp, cv.COLOR_GRAY2BGR)
    overlay[missing>0] = (255,0,0)  # red
    overlay[extra>0]   = (0,255,0)  # green
    plt.figure(figsize=(4,4))
    plt.imshow(overlay[:,:,::-1])
    plt.title(f"{fname}: {label.upper()} ({count} teeth)")
    plt.axis('off')
    plt.show()